# **Question 1**
In a bicycle-sharing system, bicycles are stored at fixed docking stations throughout the city. Users can rent
bicycles from one docking station and return them to any other docking station. These systems are often used
for short trips around the city, providing users with a convenient and eco-friendly mode of transportation. A
dataset (link) containing 6,867 bicycle trips over one day is provided. The column descriptions are provided
below.
• trip id: Unique trip identifier.
• started at: Start time of the trip
• ended at: End time of the trip
• start lat/start lng: Latitude/Longitude of the starting depot
• end lat/end lng: Latitude/Longitude of the end depot

1. Write a function that removes all trips of duration 0 minutes and prints the following values on the
console. Mention the same values in the report.
• Maximum duration of the trip (in minutes).
• Minimum duration of the trip (in minutes).
• Total number of trips corresponding to the minimum duration.
• Percentage of total circular trips. A trip is defined as circular if it starts and ends at the same
location.
• Total runtime for the function
Hint: The question is designed to judge your basic skills in exploratory data analysis.

In [ ]:
import pandas as pd
import time

In [ ]:
import pandas as pd
import time

def remove_zero_duration_trips(df):
    start_time = time.time()
    df['duration'] = (pd.to_datetime(df.loc[:, 'ended_at']) - pd.to_datetime(df.loc[:, 'started_at'])).dt.total_seconds() / 60
    df = df[df['duration'] != 0]
    max_duration = df['duration'].max()
    min_duration = df['duration'].min()
    total_trips_min_duration = len(df[df['duration'] == min_duration])
    num_circular_trips = len(df[(df['start_lat'] == df['end_lat']) & (df['start_lng'] == df['end_lng'])])
    percentage_circular_trips = (num_circular_trips / len(df)) * 100
    end_time = time.time()
    total_runtime = end_time - start_time
    print(f"Maximum duration of the trip (in minutes): {max_duration}")
    print(f"Minimum duration of the trip (in minutes): {min_duration}")
    print(f"Total number of trips corresponding to the minimum duration: {total_trips_min_duration}")
    print(f"Percentage of total circular trips: {percentage_circular_trips}")
    print(f"Total runtime for the function: {total_runtime} seconds.")
    return df.drop(columns=['duration'])

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/bike_data_new.csv")
df = remove_zero_duration_trips(df)

Maximum duration of the trip (in minutes): 518.0000000000001
Minimum duration of the trip (in minutes): 1.0000000000000002
Total number of trips corresponding to the minimum duration: 89
Percentage of total circular trips: 2.4776425744025805
Total runtime for the function: 0.2273552417755127 seconds.


2. Filter the original dataset to include only the trips starting between 06:00 AM and 06:00 PM. Find
the total number of feasible pairs of trips. Two trips, A and B, are defined as a feasible pair if they
can be served in succession by the same bicycle, i.e., if the end location of trip A is the same as the
start location of trip B and the start time of the trip B is greater than or equal to the end time of the
trip A. For example, Trip Id 1733 and 1965 are feasible. In the report, mention the total feasible pairs
of trips and runtime.
Hint: The question is designed to judge your critical and analytical thinking.

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv("/content/drive/MyDrive/bike_data_new.csv")

# Convert the "started at" and "ended at" columns to datetime format
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

# Filter the original dataset to include only the trips starting between 06:00 AM and 06:00 PM
filtered_df = df[(df['started_at'].dt.hour >= 6) & (df['started_at'].dt.hour < 18)]

# Sort the filtered dataset by the end time of each trip
filtered_df = filtered_df.sort_values(by=['ended_at'])

# Initialize a counter for the number of feasible pairs of trips
feasible_pairs_count = 0

# Loop through each trip in the filtered dataset
for i in range(len(filtered_df)):
    # Get the end location and end time of the current trip
    current_end_location = filtered_df.iloc[i]['end_lat'], filtered_df.iloc[i]['end_lng']
    current_end_time = filtered_df.iloc[i]['ended_at']
    
    # Find the next feasible trip
    next_feasible_trip = filtered_df[(filtered_df['start_lat'] == current_end_location[0]) & 
                                     (filtered_df['start_lng'] == current_end_location[1]) & 
                                     (filtered_df['started_at'] >= current_end_time)]
    
    # Increment the counter if a feasible pair of trips was found
    if len(next_feasible_trip) > 0:
        feasible_pairs_count += 1

# Print the total number of feasible pairs of trips
print(f'Total number of feasible pairs of trips: {feasible_pairs_count}')


Total number of feasible pairs of trips: 4187


3. Filter the original dataset to include only the first 100 trips (i.e., trip id 1 to 100). In the report,
mention the number of unique depots used to serve these trips. Next, find the shortest path distance
between all the depots. To do so, do the following steps:
• Download the underlying graph using the OSMnX module (reference) in Python.
• Find the nearest node in the graph corresponding to each depot.
• For every pair of nodes, run a shortest path algorithm (for example, Dijkstra’s (reference)) to
get the length of the shortest path. If the nodes are not reachable from each other, the function
should return -1.
In the report, mention the total runtime and the maximum and minimum (greater than 0) distance.

In [ ]:
!pip install osmnx

In [ ]:
!pip install -U scikit-learn

In [ ]:
import pandas as pd
import osmnx as ox
import networkx as nx
from itertools import combinations

def find_shortest_path_distance(df):
    depots = df[['start_lat', 'start_lng']].drop_duplicates()[:100] # take first 100 unique depots
    G = ox.graph_from_place('New York City, New York, USA', network_type='bike') # download bike network graph for NYC
    distances = []
    depots['node'] = ox.distance.nearest_nodes(G, depots['start_lat'], depots['start_lng']) # find nearest node for each depot
    distances = []
    for pair in combinations(depots.iterrows(), 2): # find shortest path distance for every pair of depots
        try:
            path_length = nx.shortest_path_length(G, pair[0][1]['node'], pair[1][1]['node'], weight='length')
            distances.append(path_length)
        except nx.NetworkXNoPath:
            distances.append(-1)
    return distances

In [ ]:
# Load the dataset
df = pd.read_csv("/content/drive/MyDrive/bike_data_new.csv")

find_shortest_path_distance(df)

ImportError: ignored

# Part 3 have some issues from institute side that's why I am getting error